In [1]:
import numpy as np
from scipy.optimize import curve_fit
import datetime
import time

In [2]:
def base_function(t, tau, dead_band_upper):
    return dead_band_upper*np.exp(-t/tau)

def estimate_RC(temperature):
    t = np.arange(len(temperature))
    popt, pcov = curve_fit(base_function, t, np.array(temperature))
    return popt[0]

def is_occupied(occupancy):
    return (occupancy != 0)

def control_fn(curr_time, occupied, schedule, tau, current_temp, dead_band_upper, dead_band_lower):
    if schedule[curr_time + tau] == 1:
        signal = 1
    elif current_temp < dead_band_lower:
        signal = 1
    elif occupied:
        signal = 1
    elif current_temp > dead_band_upper:
        signal = 0
    else:
        signal = 0
    return signal

def convert_to_hour(time, sampling_frequency):
    return time/(60*60*sampling_frequency)

class RC_Controller:
    def __init__(self, schedule):
        self.setpoint = 21
        self.dead_band_upper = 22
        self.dead_band_lower = 20

        self.current_temp = self.setpoint  # Assume temp at setpoint to start
        self.occupancy = 0  # Assume no occupants to start

        self.schedule = schedule
        self.hist_temperature = []

        self.sampling_frequency = 1/30
        self.tau = 10 # Initial guess of the time constant = 10hr for the room

        self.flag = 0

    def get_tau(self):
        tau = estimate_RC(self.hist_temperature)
        return tau
    
    def update(self):

        currentDateAndTime = datetime.datetime.now()
        currentTime = currentDateAndTime.hour
        currentDate = currentDateAndTime.date()
        day, hour = (currentDate.isoweekday()-1), currentTime

        curr_time = 24*day + hour
        
        if len(self.hist_temperature) > 60*(60*self.sampling_frequency): # the value of tau is only calculated and update if the hist_temperature recorded is more than an hour long
            tau = self.get_tau()
            tau = convert_to_hour(tau, self.sampling_frequency)
            self.tau = tau

        if self.current_temp >= self.dead_band_upper:
            self.flag = 1
        elif self.current_temp <= self.dead_band_lower:
            self.flag = 0

        if self.flag == 1:
            self.hist_temperature.append(self.current_temp)
        if self.flag == 0:
            self.hist_temperature = []

        occupied = is_occupied(self.occupancy)
        control = control_fn(curr_time, occupied, self.schedule, self.tau, self.current_temp, self.dead_band_upper, self.dead_band_lower)

        return control

def recv_instr():
    global controller 
    print("Waiting for client request..")
    while True:
        clientConnection, clientAddress = server.accept()
        print("Connected clinet :" , clientAddress)
        data = (clientConnection.recv(1024)).decode()
        print("From Client :" , data)
        try:
            endpoint, val = data.split()
            print(endpoint)
            if endpoint == '/temp':
                controller.current_temp = float(val)
                print("temp assigned")
            elif endpoint == '/ultrasonic':
                controller.occupancy = int(val)
                print("ultrasonic assigned")
            else:
                print("Bad endpoint")
        except IndexError:
            print("Bad data")
        clientConnection.close()


# Function to convert angle to PWM duty cycle
def angle_to_duty_cycle(angle):
    duty_cycle = (angle / 90.0) * (PWM_duty_max - PWM_duty_min) + PWM_duty_min
    return duty_cycle

# Function to move the servo to a specified angle
def move_to_angle(angle):
    assert 0 <= angle <= 90

    duty_cycle = angle_to_duty_cycle(angle)
    pwm.ChangeDutyCycle(duty_cycle)
    time.sleep(1)  # Adjust this delay as needed for your servo
    # pwm.ChangeDutyCycle(0)  # Stop sending PWM signal


In [3]:
schedule = [1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,
            1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,
            1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,
            1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,
            1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,
            1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1]
# Initialize controller as global between threads
controller = RC_Controller(schedule)

In [4]:
controller.update()

1

In [8]:
controller.current_temp, controller.tau, controller.dead_band_upper, controller.dead_band_lower

(21, 10, 22, 20)